# Experiments Notebook

In [ ]:
# Path to DICOM directory
DICOM_PATH = "/home/pyuser/data/Paradise_DICOMs"
# DICOM_PATH = "/home/pyuser/data/Paradise_Test_DICOMs"

# Path to Histogram directory
HISTOGRAM_PATH = "/home/pyuser/data/Paradise_Histograms"

# Path to Experiments directory
EXPERIMENTS_PATH = "/home/pyuser/data/Paradise_Experiments"

import os
import numpy as np
import pydicom
from PIL import Image
from pathlib import Path
import warnings

# Suppress the specific pydicom character encoding warning
warnings.filterwarnings("ignore", category=UserWarning, module="pydicom.charset")

FILES_TO_CONVERT = [28695112]

## RGB Encoding for Extended Dynamic Range

The `convert_dicom_to_rgb_png()` function implements a novel approach to preserve DICOM's high dynamic range in standard PNG format by encoding pixel values across RGB channels.

### How RGB Encoding Works:

1. **Normalization**: Pixel values are normalized to 0-1 range based on actual data range
2. **Scaling**: Values are scaled to 0-765 range (255 × 3 channels)  
3. **Distribution**: Values are distributed across RGB channels sequentially:
   - **0-255**: All in Red channel `(value, 0, 0)`
   - **256-510**: Red maxed, remainder in Green `(255, value-255, 0)`
   - **511-765**: Red and Green maxed, remainder in Blue `(255, 255, value-510)`

### Example:
- **Original pixel**: 3000 (in 0-4095 range)
- **Normalized**: 3000/4095 = 0.732
- **Scaled**: 0.732 × 765 = 560
- **RGB encoding**: `(255, 255, 50)` where 255+255+50 = 560

### Benefits:
- **3× Dynamic Range**: 766 values vs standard grayscale's 256
- **Lossless**: Perfect reconstruction possible via `decode_rgb_png_to_pixels()`
- **Standard Format**: Uses regular PNG files, compatible with any image viewer
- **MONOCHROME Handling**: Automatic conversion between MONOCHROME1 and MONOCHROME2


In [ ]:
def convert_dicom_to_rgb_png(files_to_convert, target_monochrome="MONOCHROME2"):
    """
    Convert DICOM files to PNG with RGB encoding for extended dynamic range.
    
    This function encodes pixel values across RGB channels to effectively triple
    the dynamic range compared to standard grayscale conversion.
    
    Parameters:
    -----------
    files_to_convert : list
        List of FileIDs (without extension) to convert
    target_monochrome : str, optional
        Target monochrome format ("MONOCHROME1" or "MONOCHROME2"). Default is "MONOCHROME2"
    
    Returns:
    --------
    dict : Dictionary with conversion results and statistics
    """
    
    # Create experiments directory if it doesn't exist
    Path(EXPERIMENTS_PATH).mkdir(parents=True, exist_ok=True)
    
    # Get all DICOM files in the directory
    all_dicom_files = {}
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                # Extract FileID (filename without extension)
                file_id = os.path.splitext(file)[0]
                try:
                    file_id = int(file_id)  # Convert to int if it's numeric
                except ValueError:
                    pass  # Keep as string if not numeric
                all_dicom_files[file_id] = os.path.join(root, file)
    
    conversion_results = {
        'successful': [],
        'failed': [],
        'not_found': [],
        'total_processed': 0
    }
    
    print(f\"Starting conversion of {len(files_to_convert)} DICOM files...\")\n    print(f\"Target monochrome format: {target_monochrome}\")\n    print(f\"Output directory: {EXPERIMENTS_PATH}\\n\")\n    \n    for file_id in files_to_convert:\n        if file_id not in all_dicom_files:\n            print(f\"File {file_id} not found in DICOM directory\")\n            conversion_results['not_found'].append(file_id)\n            continue\n            \n        dicom_path = all_dicom_files[file_id]\n        \n        try:\n            # Read DICOM file\n            ds = pydicom.dcmread(dicom_path)\n            \n            # Get pixel data\n            pixel_array = ds.pixel_array.astype(np.float64)\n            \n            # Get photometric interpretation\n            photometric = getattr(ds, 'PhotometricInterpretation', 'MONOCHROME2')\n            \n            # Convert MONOCHROME1 to MONOCHROME2 if needed\n            if photometric == 'MONOCHROME1' and target_monochrome == 'MONOCHROME2':\n                # In MONOCHROME1, higher values = darker, so invert\n                max_possible = np.max(pixel_array)\n                pixel_array = max_possible - pixel_array\n                print(f\"  Converted {file_id} from MONOCHROME1 to MONOCHROME2\")\n            elif photometric == 'MONOCHROME2' and target_monochrome == 'MONOCHROME1':\n                # In MONOCHROME2, higher values = brighter, so invert\n                max_possible = np.max(pixel_array)\n                pixel_array = max_possible - pixel_array\n                print(f\"  Converted {file_id} from MONOCHROME2 to MONOCHROME1\")\n            \n            # Get actual bit depth based on pixel values\n            min_pixel = np.min(pixel_array)\n            max_pixel = np.max(pixel_array)\n            \n            # Calculate dynamic range\n            pixel_range = max_pixel - min_pixel\n            \n            if pixel_range == 0:\n                print(f\"  Warning: {file_id} has uniform pixel values\")\n                # Handle uniform images\n                normalized_array = np.zeros_like(pixel_array)\n            else:\n                # Normalize to 0-1 range\n                normalized_array = (pixel_array - min_pixel) / pixel_range\n            \n            # Scale to RGB range (0 to 255*3 = 765)\n            rgb_range = 255 * 3\n            scaled_array = normalized_array * rgb_range\n            \n            # Convert to RGB encoding\n            height, width = pixel_array.shape\n            rgb_array = np.zeros((height, width, 3), dtype=np.uint8)\n            \n            # Distribute values across RGB channels\n            for i in range(height):\n                for j in range(width):\n                    value = scaled_array[i, j]\n                    \n                    # Distribute across R, G, B channels\n                    if value <= 255:\n                        # All in R channel\n                        rgb_array[i, j] = [int(value), 0, 0]\n                    elif value <= 510:\n                        # R maxed, remainder in G\n                        rgb_array[i, j] = [255, int(value - 255), 0]\n                    else:\n                        # R and G maxed, remainder in B\n                        rgb_array[i, j] = [255, 255, int(value - 510)]\n            \n            # Create PIL Image\n            pil_image = Image.fromarray(rgb_array, 'RGB')\n            \n            # Save as PNG\n            output_filename = f\"{file_id}_rgb_encoded.png\"\n            output_path = os.path.join(EXPERIMENTS_PATH, output_filename)\n            pil_image.save(output_path)\n            \n            # Store conversion info\n            conversion_info = {\n                'file_id': file_id,\n                'original_photometric': photometric,\n                'target_photometric': target_monochrome,\n                'original_range': (min_pixel, max_pixel),\n                'dynamic_range': pixel_range,\n                'output_file': output_filename\n            }\n            \n            conversion_results['successful'].append(conversion_info)\n            conversion_results['total_processed'] += 1\n            \n            print(f\"  ✓ {file_id}: {photometric} → PNG (range: {min_pixel:.0f}-{max_pixel:.0f})\")\n            \n        except Exception as e:\n            print(f\"  ✗ Error processing {file_id}: {str(e)}\")\n            conversion_results['failed'].append({'file_id': file_id, 'error': str(e)})\n    \n    # Print summary\n    print(f\"\\n=== CONVERSION SUMMARY ===\")\n    print(f\"Successfully converted: {len(conversion_results['successful'])} files\")\n    print(f\"Failed conversions: {len(conversion_results['failed'])} files\")\n    print(f\"Files not found: {len(conversion_results['not_found'])} files\")\n    print(f\"Total processed: {conversion_results['total_processed']} files\")\n    \n    if conversion_results['successful']:\n        print(f\"\\nOutput files saved to: {EXPERIMENTS_PATH}\")\n        for result in conversion_results['successful']:\n            print(f\"  - {result['output_file']}\")\n    \n    return conversion_results\n\n# Execute the conversion\nresults = convert_dicom_to_rgb_png(FILES_TO_CONVERT)


In [ ]:
def decode_rgb_png_to_pixels(png_path):
    """
    Decode an RGB-encoded PNG back to original pixel values.
    
    This function reverses the RGB encoding process to recover the original
    pixel values with their full dynamic range.
    
    Parameters:
    -----------
    png_path : str
        Path to the RGB-encoded PNG file
    
    Returns:
    --------
    numpy.ndarray : Decoded pixel array with values in 0-765 range
    """
    
    # Load the PNG image
    pil_image = Image.open(png_path)
    rgb_array = np.array(pil_image)
    
    # Decode RGB channels back to pixel values
    height, width, _ = rgb_array.shape
    decoded_array = np.zeros((height, width), dtype=np.float64)
    
    for i in range(height):
        for j in range(width):
            r, g, b = rgb_array[i, j]
            # Reconstruct the original scaled value
            decoded_value = r + g + b
            decoded_array[i, j] = decoded_value
    
    return decoded_array

def demonstrate_rgb_encoding():
    """
    Demonstrate the RGB encoding with a simple example.
    Shows how different pixel values are encoded across RGB channels.
    """
    print("=== RGB ENCODING DEMONSTRATION ===")
    print("How pixel values are distributed across RGB channels:")
    print()
    
    test_values = [0, 100, 255, 300, 510, 600, 765]
    
    for value in test_values:
        if value <= 255:
            rgb = [value, 0, 0]
        elif value <= 510:
            rgb = [255, value - 255, 0]
        else:
            rgb = [255, 255, value - 510]
        
        reconstructed = sum(rgb)
        print(f"Value {value:3d} → RGB({rgb[0]:3d}, {rgb[1]:3d}, {rgb[2]:3d}) → Reconstructed: {reconstructed}")
    
    print()
    print("This encoding allows us to represent 766 different values (0-765)")
    print("compared to standard grayscale's 256 values (0-255)")
    print("Effective dynamic range increase: 3x")

# Run the demonstration
demonstrate_rgb_encoding()


In [ ]:
# Additional usage examples and batch processing

def convert_multiple_file_lists():
    """
    Example showing how to convert different sets of files with different parameters.
    """
    
    # Example 1: Convert specific files to MONOCHROME2 (default)
    files_set_1 = [28695112, 28695113, 28695114]  # Add more FileIDs as needed
    print("=== CONVERTING SET 1 TO MONOCHROME2 ===")
    results_1 = convert_dicom_to_rgb_png(files_set_1)
    
    # Example 2: Convert to MONOCHROME1 format
    files_set_2 = [28695115, 28695116]  # Add more FileIDs as needed
    print("\n=== CONVERTING SET 2 TO MONOCHROME1 ===")
    results_2 = convert_dicom_to_rgb_png(files_set_2, target_monochrome="MONOCHROME1")
    
    return results_1, results_2

def analyze_conversion_results(results):
    """
    Analyze and display detailed information about conversion results.
    """
    if not results['successful']:
        print("No successful conversions to analyze.")
        return
    
    print("=== DETAILED CONVERSION ANALYSIS ===")
    
    for result in results['successful']:
        print(f"\nFile: {result['file_id']}")
        print(f"  Original format: {result['original_photometric']}")
        print(f"  Target format: {result['target_photometric']}")
        print(f"  Original range: {result['original_range'][0]:.0f} - {result['original_range'][1]:.0f}")
        print(f"  Dynamic range: {result['dynamic_range']:.0f}")
        print(f"  Output file: {result['output_file']}")
        
        # Calculate compression ratio (theoretical)
        original_values = result['dynamic_range'] + 1
        rgb_values = 766  # 0-765 range
        compression_info = "Preserved" if rgb_values >= original_values else f"Compressed from {original_values:.0f} to {rgb_values}"
        print(f"  Dynamic range handling: {compression_info}")

# Example usage with current FILES_TO_CONVERT list
print("=== CURRENT CONVERSION TASK ===")
print(f"Files to convert: {FILES_TO_CONVERT}")
print(f"Target directory: {EXPERIMENTS_PATH}")
print()

# Uncomment the following lines to run different conversion examples:

# Run the main conversion with current FILES_TO_CONVERT
results = convert_dicom_to_rgb_png(FILES_TO_CONVERT)
# analyze_conversion_results(results)

# Run multiple conversions with different parameters
# results_1, results_2 = convert_multiple_file_lists()

print("Ready to run conversions!")
print("Uncomment the desired lines above to execute the conversion functions.")


In [ ]:
# Function to convert a DICOM file to a PNG file while encoding pixel value in the 3 chanels (RGB)

# Example: If pixel value is 3000 when the max possible would be 4095, then the RGB values would be calculated as follows:
# - Pixel value = 3000 / 4095
# - New pixel value = 3000 / 4095 * (255 * 3) = 560
# - RGB values = (255, 255, 50)  # The sum of all 3 gives us the value of the pixel, effectively giving us 3x more possible values for pixel values